# Day 2. Word Embeddings


In this notebook, we will be using the Stanford's pre-trained embedding model **GloVe**. **GloVe** is trained based on the global word-word co-occurrence statistics from a corpus. Researchers were amazed with the linear structures that emerge from the resulting vector space. With this notebook, we hope to showcase you some of the interesting and exciting phenomena related to these word embeddings.  


## Agenda

- Visualizing word vector representations
- Word similarity using word embeddings
- Word Analogies

![Schematics of word embeddigns and their representation in lower dimensional spaces.](https://miro.medium.com/max/1400/1*sAJdxEsDjsPMioHyzlN3_A.png)



## [3 min] Setup 

In this section, we will install and import the necessary libraries for the execution of this notebook.

In [ ]:
# Dimensionality reduction
from sklearn.decomposition import PCA

# Word embeddings loading
from gensim.models import KeyedVectors
import gensim.downloader

# Data processing
import numpy as np

# Data visualization
import matplotlib.pyplot as plt

# Interactive plotting
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "simple_white"

We will use the word embeddings that are already available out-of-the-box in the `gensim` library. With the following command, we list all the embeddings that are available for download.

In [ ]:
# Show all available models in gensim
print("List of available models:\n\n ->", "\n -> ".join(list(gensim.downloader.info()["models"].keys())))

List of available models:

 -> fasttext-wiki-news-subwords-300
 -> conceptnet-numberbatch-17-06-300
 -> word2vec-ruscorpora-300
 -> word2vec-google-news-300
 -> glove-wiki-gigaword-50
 -> glove-wiki-gigaword-100
 -> glove-wiki-gigaword-200
 -> glove-wiki-gigaword-300
 -> glove-twitter-25
 -> glove-twitter-50
 -> glove-twitter-100
 -> glove-twitter-200
 -> __testing_word2vec-matrix-synopsis


## [5 min] Load Word Embeddings 

In this notebook, we will be using the Stanford's pre-trained embedding model - **GloVe**. **GloVe** is trained based on the global word-word co-occurrence statistics from a corpus. Researchers were amazed with the linear structures that emerge from the resulting vector space. With this notebook, we hope to showcase you some of the interesting and exciting phenomena related to these word embeddings.  

If you'd like to checkout more about the **GloVe** project, check out their [Official Webpage](https://nlp.stanford.edu/projects/glove/).

**Downloading the *GloVe* representations?** 

We will use the 50-dimensional **GloVe** vector representation. Since `gensim`, a free open-source Python library for efficiently and effortlessly representing documents as semantic vectors, already provides different **GloVe** models, we will download the 50-dimensional version: `glove-wiki-gigaword-50`.

(**Note**: The download may take up to 2 min to conclude.)

In [ ]:
%%time
embeddings = gensim.downloader.load("glove-wiki-gigaword-50")

CPU times: user 14.6 s, sys: 482 ms, total: 15.1 s
Wall time: 15.2 s


In [ ]:
vocab_words = list(embeddings.vocab.keys())
print("Number of words in vocabulary:", len(vocab_words))

Number of words in vocabulary: 400000


### Helper functions

In the next couple of cells, we will define a set of functions that we will use throughout the notebook.

In [ ]:
def train_dim_reducer_for_words(embeddings, words: list, seed: int=12938):
  """Train a dimensionality reduction algorithm for subset of words."""
  rand = np.random.default_rng(seed)

  # Retrieve the vector representation for each sampled word
  matrix = np.array([embeddings[w] for w in words])

  # Initialize dimensionality reduction to 3 dimensions (`n_components=3`)
  pca = PCA(n_components=3, random_state=rand.integers(0 , 10**6), whiten=True)
  # Learn projections to lower dimension
  pca.fit(matrix)
  print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())

  return pca

def train_dim_reducer(embeddings, frac: float=1.0, seed: int=12938):
  """Train a dimensionality reduction algorithm using the whole 
  vocabulary representations.
  """
  rand = np.random.default_rng(seed)

  vocab = embeddings.vocab.keys()
  
  if frac < 1.0:
    n_samples = int(min(frac * len(vocab), len(vocab)))

    # Sample `frac` words from the vocabulary
    words = list(rand.choice(vocab, size=n_samples, replace=False))
  else:
    words = vocab

  return train_dim_reducer_for_words(embeddings, words, seed)

In the next cell, we define two methods for displaying interactive plots of the embeddings:

- `plot_word_embeddings2d`: creates 2-dimensional interactive plots of the list of words provided by the user.
- `plot_word_embeddings3d`: creates 3-dimensional interactive plots of the words provided by the user.

In [ ]:
def plot_word_embeddings2d(embeddings, dim_reducer, *words_lists: list):
  # Flatten list into single list
  all_words = []
  for lst in words_lists:
    all_words.extend(lst)  
  
  # `emb_matrix` consists of word embeddings stacked vertically.
  # the matrix has shape |W| x |E|, where
  # - |W| is the number of words in `words`
  # - |E| is the dimensionality of the word embeddings
  embeddings_matrix = np.array([embeddings[w] for w in all_words])
  embeddings_matrix = dim_reducer.transform(embeddings_matrix)
    
  # ---------------------------------------------
  # Create the plot
  # ---------------------------------------------
  count = 0
  traces = []

  for words in words_lists:
    size = len(words)

    trace = go.Scatter(
      x = embeddings_matrix[count:count+size,0], 
      y = embeddings_matrix[count:count+size,1], 
      text = words,
      name = words[0],
      textposition = "top center",
      textfont_size = 20,
      mode = 'markers+text',
      marker = {
          'size': 10,
          'opacity': 0.8,
          'color': 2
    })

    count += size
    traces.append(trace)

  # Configure the layout
  layout = go.Layout(
      margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
      showlegend=True,
      legend=dict(
      x=1,
      y=0.5,
      font=dict(
          family="Courier New",
          size=25,
          color="black"
      )),
      font = dict(
          family = "Courier New",
          size = 15),
      autosize = True,
    )

  plot_figure = go.Figure(data = traces, layout = layout)
  plot_figure.update_yaxes(showgrid=True)
  plot_figure.update_xaxes(showgrid=True)
  plot_figure.show()


def plot_word_embeddings3d(embeddings, dim_reducer, *words_lists: list):
  # Flatten list into single list
  all_words = []
  for lst in words_lists:
    all_words.extend(lst)  
  
  # `emb_matrix` consists of word embeddings stacked vertically.
  # the matrix has shape |W| x |E|, where
  # - |W| is the number of words in `words`
  # - |E| is the dimensionality of the word embeddings
  embeddings_matrix = np.array([embeddings.get_vector(w) for w in all_words])
  embeddings_matrix = dim_reducer.transform(embeddings_matrix)
    
  # ---------------------------------------------
  # Create the plot
  # ---------------------------------------------
  count = 0
  traces = []

  for words in words_lists:
    size = len(words)

    trace = go.Scatter3d(
      x = embeddings_matrix[count:count+size,0], 
      y = embeddings_matrix[count:count+size,1], 
      z = embeddings_matrix[count:count+size,2], 
      text = words,
      name = words[0],
      textposition = "top center",
      textfont_size = 10,
      mode = 'markers+text',
      marker = {
          'size': 6,
          'opacity': 0.8,
          'color': 2
    })

    count += size
    traces.append(trace)

  # Configure the layout
  layout = go.Layout(
      margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
      showlegend=True,
      legend=dict(
      x=1,
      y=0.5,
      font=dict(
          family="Courier New",
          size=20,
          color="black"
      )),
      font = dict(
          family = "Courier New",
          size = 15),
      autosize = True,
    )

  axis_kwargs = dict(
    showgrid=True,
    gridcolor="lightgray",
    showbackground=False,
    zerolinecolor="black",
  )
  plot_figure = go.Figure(data = traces, layout = layout)
  plot_figure.update_layout(
      scene=dict(xaxis = axis_kwargs, yaxis = axis_kwargs, zaxis = axis_kwargs))
  plot_figure.show()

Now, we will define a method to return the most similar words to a given set of words. This is just a wrapper around the implementation's method to more easily handle scores and the words.

Positive words contribute positively towards the similarity, negative words negatively.

If you provide a list, this method computes cosine similarity between a simple mean of the projection weight vectors of the given words and the vectors for each word in the model.

In [ ]:
def most_similar(embeddings, positive=None, negative=None, with_scores: bool=False, topn=5):
  """Computes the most similar words to a given set of words. 
  
  Positive words contribute positively towards the similarity, negative words
  negatively.

  If you provide a list of words, this method computes cosine similarity
  between a simple mean of the projection weight vectors of the given words
  and the vectors for each word in the model.

  Notes
  -----
  This is just a wrapper around the implementation's method to more easily
  handle scores and the words.
  """
  assert positive is not None or negative is not None

  words = embeddings.most_similar(positive=positive, negative=negative, topn=topn) 
  words, scores = zip(*words)
  if with_scores:
    return words, scores
  else:
    return words


def print_words(words, dec: int=4):
  """Hacky version: not pretty, do not copy."""
  if isinstance(words[1], (tuple, list, float)):
    for (word, score) in zip(*words):
      print("-->", word, "\t", round(score, dec))
  else:
    for word in words:
      print("-->", word)

The following method computes word analogies according to the following rule:

```
word4 = word3 + (word2 - word1)
```

As we will see later on, this will help us retrieve relationships of the form: `'word1' is to 'word2', what 'word3' is to 'word4'`. As an example, you have: 
`man is to woman, what king is to ______`.

In [ ]:
def find_analogy(word1: str, word2: str, word3: str, embeddings=embeddings, with_scores: bool=True, n=2):
  """Computes the analogy based on the operation word4 = word3 + (word2 - word1)."""

  # For each word, get their word embeddings
  emb1 = embeddings[word1]
  emb2 = embeddings[word2]
  emb3 = embeddings[word3]

  result = np.array([-emb1 + emb2 + emb3])
  result, scores = most_similar(embeddings, result, with_scores=True, topn=n)

  for r, score in zip(result, scores):
    message = f"'{word1}' is to '{word2}', what '{word3}', is to ______?"
    message += f" ---> {r}"

    if with_scores:
      message += f"\t{round(score, 4)}"

    print(message)

## [5 min] Visualizing Word Embeddings

Now that we have the embeddings model, we can map words into a vector space and visualize them.

We've implemented two methods that allow you to visualize a lower dimensional representation of the words embeddings. `plot_word_embeddings2d` plots 2D representations, whereas `plot_word_embeddings3d` plots 3D representations. 

As the inputs, we will need to provide the following:

- `embeddings`: the word vector representation that we're using. In this case, the GloVe embeddings object that we've downloaded previously. 

- `dim_reducer`: a model that is able to transform our word embeddings' high-dimensional representations into low-dimensional representations. We can obtain that either through `train_dim_reducer` or `train_dim_reducer_for_words`.

- `list_of_words`: you can pass a single list of words, or multiple lists of words that you wanna plot. The main difference is that per each list you pass as an argument, there should be a different color.

In [ ]:
# Define a set of words to plot:
list1 = ['school', 'science', 'college', 'university']
list2 = ['fruits', 'banana', 'apple', 'tomato']

# Create the dimensionality reductor object
dim_reducer = train_dim_reducer(embeddings)

# Plot as many list of words as you'd like
plot_word_embeddings2d(embeddings, dim_reducer, list1, list2)

Variance explained: 0.13


In [ ]:
plot_word_embeddings3d(embeddings, dim_reducer, list1, list2)

**Alternatively**: If your representations are not making too much sense, you can opt for reducing the dimensions of only a subset of the words (so that the dimensionality reduction algorithm focuses only on the properties of those word representations).

In [ ]:
# Define a set of words to plot:
list1 = ['school', 'science', 'college', 'university']
list2 = ['fruits', 'banana', 'apple', 'tomato']

# Create the dimensionality reductor object
dim_reducer2 = train_dim_reducer_for_words(embeddings, list1 + list2)

# Plot as many list of words as you'd like
plot_word_embeddings2d(embeddings, dim_reducer2, list1, list2)

Variance explained: 0.84


### Exercise 

Go ahead an play around with the list of words. You can add as many as you'd like. Note that if you end up selecting words that do not exist you might face errors. **Find some interesting relationships between different words.**

In [ ]:
list1 = ... # TODO: define your own list of words 
list2 = ... # TODO: define your own list of words
# ... Add as many lists as you want!

# You can either re-use the previously created dimensionality reduction object
# or create your own, depending on the list of words you specify
# dim_reducer = train_dim_reducer_for_words(embeddings, list1 + list2)

plot_word_embeddings2d(embeddings, dim_reducer, list1, list2)

## [5 min] Discovering similar words using Word Embeddings


We can use the `embeddings` to find the most similar words to either a single word or a list of words! Let's try it out!

In [ ]:
# Number of top similar words to retrieve
topn = 5

# -----------------------------------------------
# Find similar words to a single `word`
# -----------------------------------------------
word = "summer"
print("Looking for the top", topn, "most similar words to:", word)
print_words(most_similar(embeddings, word, topn=topn, with_scores=True))
print()

# -----------------------------------------------
# Find similar words to a list
# -----------------------------------------------
list_of_words = ["summer", "winter"]
print("Looking for the top", topn, "most similar words to:", list_of_words)
print_words(most_similar(embeddings, list_of_words, topn=topn, with_scores=True))
print()

# -----------------------------------------------
# Find similar words to a list of words but 
# not as similar to a seconf list of words.
# -----------------------------------------------
list_of_pos_words = ["summer", "winter"]
list_of_neg_words = ["spring", "autumn"]

print("List of positive words:", list_of_pos_words)
print("List of negative words:", list_of_neg_words)
print_words(most_similar(embeddings, list_of_pos_words, list_of_neg_words, topn=topn, with_scores=True))

Looking for the top 5 most similar words to: summer
--> winter 	 0.92
--> spring 	 0.8946
--> autumn 	 0.8394
--> beginning 	 0.8159
--> starting 	 0.7925

Looking for the top 5 most similar words to: ['summer', 'winter']
--> spring 	 0.8966
--> autumn 	 0.8601
--> beginning 	 0.7695
--> day 	 0.7614
--> rainy 	 0.7606

List of positive words: ['summer', 'winter']
List of negative words: ['spring', 'autumn']
--> braugher 	 0.5727
--> skier 	 0.5636
--> snowboarders 	 0.5467
--> skiers 	 0.546
--> athlete 	 0.5434


If you do not want to observe the similarity scores? Just remove the flag `with_scores=True` or set it to `False`. 

In [ ]:
# Number of top similar words to retrieve
topn = 5

# -----------------------------------------------
# Find similar words to a single `word`
# -----------------------------------------------
word = "summer"
print("Looking for the top", topn, "most similar words to:", word)
print_words(most_similar(embeddings, word, topn=topn, with_scores=True))

Looking for the top 5 most similar words to: summer
--> winter 	 0.92
--> spring 	 0.8946
--> autumn 	 0.8394
--> beginning 	 0.8159
--> starting 	 0.7925


### Exercise 1.

Play around with the `most_similar` method and find relationships between different pairs of words. Do these make sense to you?


You can also use the visualization methods to help you understand better how closely related the words are.

In [ ]:
list_words = ... # TODO:

print(most_similar(embeddings, list_words, topn=topn))

### Exercise 2. 

Find interesting mistakes concerning the word embeddings. What was the result you obtained and what were you expecting it to be?

In [ ]:
list_words = ... # TODO:

print(most_similar(embeddings, list_words, topn=topn))

In the following sections, we will focus on different types of analogies using word embeddings. We will see how we can leverage the vector space and the vector operations to verify different relationships. 

## [20 min] Word Analogies

We have seen before that word embeddings are nothing but a dense vector representation. When researchers developed these models, they observed interesting linear substructures in this vector space. 

For example, using vector addition and subtraction, they were able to find relationships between sets of words. The most popularized of which is:

> ***man* is to *woman* what *king* is to ____.**

Naturally, we expect the answer to be **queen**, but is it? Let's try it out. 

In [ ]:
find_analogy("man", "woman", "king", embeddings=embeddings, n=3)

'man' is to 'woman', what 'king', is to ______? ---> king	0.886
'man' is to 'woman', what 'king', is to ______? ---> queen	0.861
'man' is to 'woman', what 'king', is to ______? ---> daughter	0.7685


The method `find_analogy` performs the following operation: 

`word4` $\approx$ (`word2` - `word1`) + `word3`, which in this case becomes `queen` $\approx$ (`woman` - `man`) + `king`.


In [ ]:
find_analogy("boy", "girl", "man", embeddings=embeddings)

'boy' is to 'girl', what 'man', is to ______? ---> woman	0.9389
'boy' is to 'girl', what 'man', is to ______? ---> man	0.9316


In [ ]:
find_analogy("paris", "france", "berlin", embeddings=embeddings)

'paris' is to 'france', what 'berlin', is to ______? ---> germany	0.9215
'paris' is to 'france', what 'berlin', is to ______? ---> denmark	0.816


### Exercise 1.

Find other examples of analogies that have not been covered yet.


In [ ]:
word1 = ... # TODO
word2 = ... # TODO
word3 = ... # TODO
find_analogy(word1, word2, word3, embeddings=embeddings)

### Exercise 2.

Can you find any analogy that reflect some biases in the model?


(**hint**: what relationship can you find concerning the words `nurse`, `doctor` and `she`)? 


In [ ]:
word1 = ... # TODO
word2 = ... # TODO
word3 = ... # TODO
find_analogy(word1, word2, word3, embeddings=embeddings)

'nurse' is to 'doctor', what 'he', is to ______? ---> he	0.91
'nurse' is to 'doctor', what 'he', is to ______? ---> himself	0.8635


### Exercise 3. 

Find examples of analogies using verb tenses. Do you expect these to be captured by the model? 

### Exercise 4. 

What happens if you visualize some of these words in a lower dimensional space? Do you expect some of them to be near each other or farther apart?

### Trying your own analogies

If you want to try out your own vector operations, try following this recipe:

In [ ]:
# To do any analogies you desire, you can proceed as follows:
# Step 1. Get the word vector reprensentation
word1 = "king"
rep1 = embeddings[word1]

word2 = "man"
rep2 = embeddings[word2]

# Step2. Specify the operation between the words 
rep = rep1 + rep2
result = np.array([rep])

print_words(most_similar(embeddings, result, with_scores=True, topn=5))

## [Optionally] Working with embeddings on other languages... 

All our experimentes so far considered *English* as the main language. However, one question you might have is whether these relationships also verify for other languages. 

In this section, we would like you to exploit the embeddings available in other languages. To do so, we present you with two options:

1. Pick **monolingual** word vector representation: these were trained on other languages from scratch and might lead to completely different relationships.

2. Pick **cross-languages** word vector representation: these are given words in two different languages and trained to represent the two languages in the same vector space.


If you'd like to pick a **monolingual** you can keep reading the next section: **Working with monolingual embeddings**. However, if you prefer the **cross-lingual** version, you should skip to the next session **Working with cross-lingual embeddings**.

### Working with Monolingual Embeddings

If you know multiple languages, you might be wondering what relationships word embeddings in your language exhibit. 

1. Browse [this list of pretrained vectors](https://fasttext.cc/docs/en/pretrained-vectors.html) and identify the language you want to explore.
2. Copy the **text** link and assign it to the variable **url**. We have an example in the code cell below.
3. Load the vector representation into a Python variable (we'll use `gensim.KeyedVectors.load_word2vec_format` for that.

In [ ]:
url = ... # TODO: Paste the URL here

## Example: If we were to consider portuguese (pt) embeddings, 
## the url would be:
## url = "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pt.vec"

In [ ]:
%%time
# downloads the specified url and creates a file named "wiki.vec"
!curl -Lo wiki.vec {url}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1488M  100 1488M    0     0  23.5M      0  0:01:03  0:01:03 --:--:-- 26.4M


In [ ]:
# Load the embeddings
embeddings_lang = KeyedVectors.load_word2vec_format("wiki.vec")
print("Size of the vocabulary:", len(embeddings_lang.vocab))

Size of the vocabulary: 592108



#### Exercise

Do the relationships you found previously verify for a different language (e.g., spanish or chinese)? Can you find new patterns on a different language? 

In [ ]:
word = ... # TODO: define a word in the language you chose
most_similar(embeddings_lang, word)

### 2. Working with cross-lingual embeddings 


1. Browse to this [list of pretrained vectors](https://github.com/facebookresearch/MUSE#multilingual-word-embeddings) and identify the language you want to explore. These multilingual word embeddings will contain most words of the language you pick, as well as some words of other languages.
2. Copy the **text** link and assign it to the variable **url**. We have an example in the code cell below, where we pick the Portuguese multilingual embeddings.

In [ ]:
%%time
%%capture
# TODO: Change this URL with the one you copy from
# https://github.com/facebookresearch/MUSE#multilingual-word-embeddings
url = "https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.pt.vec"

CPU times: user 2.42 ms, sys: 42 µs, total: 2.47 ms
Wall time: 2.48 ms


In [ ]:
%%time
%%capture
# Retrieve the specified multilingual vector space
!curl -Lo wiki.multi.vec {url}

# Load the embeddings
embeddings_multi_lang = KeyedVectors.load_word2vec_format("wiki.multi.vec")
print("Size of the vocabulary:", len(embeddings_multi_lang.vocab))

CPU times: user 35.2 s, sys: 1.46 s, total: 36.6 s
Wall time: 54.1 s


Use the methods explored earlier in the class to study the relationships in your selected language. We added some examples in Portuguese... Feel free to modify them as you please!


You'll probably notice that the english representations in these cross-lingual representations are not as strong as the multilingual.

In [ ]:
print_words(most_similar(embeddings_multi_lang, "hello", with_scores=True))
print()
print_words(most_similar(embeddings_multi_lang, "olá", with_scores=True))

--> s/mileage 	 0.5829
--> musume 	 0.56
--> berryz 	 0.5411
--> goodbye 	 0.531
--> project 	 0.5243

--> belanidia 	 0.6121
--> gostaria 	 0.5944
--> desculpe 	 0.594
--> criei 	 0.5937
--> prezados 	 0.5904


In [ ]:
print_words(most_similar(embeddings_multi_lang, "friend", with_scores=True))
print()
print_words(most_similar(embeddings_multi_lang, "amigo", with_scores=True))

--> friends 	 0.7429
--> boyfriend 	 0.7327
--> friendly 	 0.656
--> girlfriend 	 0.6395
--> everyone 	 0.6392

--> namorador 	 0.6774
--> amigos 	 0.6656
--> companheiro 	 0.6611
--> colega 	 0.6526
--> admirador 	 0.6438


In [ ]:
print_words(most_similar(embeddings_multi_lang, ["language", u"língua"], with_scores=True))

--> mwlanguage 	 0.7947
--> languages 	 0.7731
--> línguagem 	 0.7462
--> interlíngua 	 0.7234
--> falada 	 0.7188


In [ ]:
find_analogy("homem", "rei", "mulher", embeddings_multi_lang)

'homem' is to 'rei', what 'mulher', is to ______? ---> rei	0.7485
'homem' is to 'rei', what 'mulher', is to ______? ---> rainha	0.6603


In [ ]:
find_analogy("homem", "enfermeiro", "mulher", embeddings_multi_lang)

'homem' is to 'enfermeiro', what 'mulher', is to ______? ---> enfermeiro	0.774
'homem' is to 'enfermeiro', what 'mulher', is to ______? ---> enfermeira	0.7595


In [ ]:
find_analogy("comer", "comi", "beber", embeddings_multi_lang)

comer is to comi what beber is to ______?
--> bebi


In [ ]:
src_words = ['man', 'king', 'woman', 'queen', 'eat', 'ate']
tgt_words = ['homem', 'rei', 'mulher', 'rainha',  'comer', 'comi']

dim_reducer = train_dim_reducer_for_words(embeddings_multi_lang, src_words + tgt_words)
plot_word_embeddings2d(embeddings_multi_lang, dim_reducer, src_words, tgt_words)

In [ ]:
src_words = ['car', 'cat', 'deep', 'feline']
tgt_words = ['carro', 'gato', 'gata', 'profundo', 'felino']

dim_reducer = train_dim_reducer_for_words(embeddings_multi_lang, src_words + tgt_words)
plot_word_embeddings3d(embeddings_multi_lang, dim_reducer, src_words, tgt_words)

# Resources

This notebook is inspired on:

- [How to add large datasets to google drive](https://www.aboutdatablog.com/post/how-to-successfully-add-large-data-sets-to-google-drive-and-use-them-in-google-colab) by Magdalena Kokiewicz.
- [Visualizing Word Embedding with PCA and t-SNE](https://towardsdatascience.com/visualizing-word-embedding-with-pca-and-t-sne-961a692509f5) by Ruben Winastwan.
- [Looking up word analogies using GloVe along the Fun-Boring axis](https://towardsdatascience.com/glove-fun-boring-664fe0717c4c) by Ludi Rehak
- [Solving Analogies by representing Words as Vectors](https://medium.datadriveninvestor.com/solving-analogies-using-word2vec-13b9e01eca1) by Rahul Rathi
- [Wikipedia2Vec](https://wikipedia2vec.github.io/wikipedia2vec/pretrained/)